In [1]:
import sys
sys.path.append('../src/')
import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

<module 'auto_co2' from '/home/mln/GIT/car-co2-prediction_pre-release/src/auto_co2/__init__.py'>

In [2]:
print(sys.path) 

['/home/mln/GIT/car-co2-prediction_pre-release/notebooks', '/home/mln/miniconda3/envs/co2/lib/python39.zip', '/home/mln/miniconda3/envs/co2/lib/python3.9', '/home/mln/miniconda3/envs/co2/lib/python3.9/lib-dynload', '', '/home/mln/miniconda3/envs/co2/lib/python3.9/site-packages', '/home/mln/GIT/car-co2-prediction_pre-release/src', '../src/']


In [3]:
df = pd.read_csv('../data/sample/co2_raw_sample.csv')


In [4]:
from auto_co2 import data
print(dir(data))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'clean_manufacturer_columns', 'co2_grade_discretization', 'column_remover', 'conditional_column_update', 'convert_dtypes', 'correct_fueltype', 'dataviz_preprocessing', 'datetime', 'download_and_load_co2_data', 'download_co2_data', 'drop_irrelevant_columns', 'drop_residual_incomplete_rows', 'dummify', 'dummify_all_features', 'electricrange_discretization', 'get_classification_data', 'kaggle', 'load_co2_data', 'ml_preprocessing', 'os', 'pd', 'pickle', 'remove_fueltype', 'rename_columns', 'save_model', 'save_processed_data', 'save_shap_values', 'select_countries', 'standardize_innovtech', 'zipfile']


In [5]:
from auto_co2.data import ml_preprocessing
#from auto_co2.data import dataviz_preprocessing
#df = dataviz_preprocessing(df)
df = co2.data.dataviz_preprocessing(df)

In [6]:
country_stats = {
 'IE': {'population': 5123536, 'gdp_per_capita': 103296.79936336935},
 'HU': {'population': 9603634, 'gdp_per_capita': 18.935334502543515},
 'ES': {'population': 47415750, 'gdp_per_capita': 30103.513733200467},
 'BE': {'population': 11584008, 'gdp_per_capita': 51785.18523122567},
 'LU': {'population': 660809, 'gdp_per_capita': 129396.30639715864},
 'FI': {'population': 5563970, 'gdp_per_capita': 53269.09118201572},
 'SE': {'population': 10548336, 'gdp_per_capita': 60375.042698298574},
 'DK': {'population': 5827463, 'gdp_per_capita': 68349.34391930074},
 'PL': {'population': 38382576, 'gdp_per_capita': 17929.40124589345},
 'LT': {'population': 2860002, 'gdp_per_capita': 24592.39504308039},
 'IT': {'population': 58850717, 'gdp_per_capita': 35927.442598770715},
 'AT': {'population': 8979894, 'gdp_per_capita': 53493.772186286384},
 'GR': {'population': 10482487, 'gdp_per_capita': 20898.272753975274},
 'PT': {'population': 10347892, 'gdp_per_capita': 24544.40455804912},
 'FR': {'population': 67749632, 'gdp_per_capita': 43658.97897812936},
 'GB': {'population': 67326569, 'gdp_per_capita': 46378.1296493068},
 'DE': {'population': 83149300, 'gdp_per_capita': 51232.36048676297},
 'EE': {'population': 1373101, 'gdp_per_capita': 27748.00466899376},
 'LV': {'population': 1883008, 'gdp_per_capita': 21855.4104193928},
 'CZ': {'population': 10827529, 'gdp_per_capita': 23152.189202171612},
 'SK': {'population': 5449270, 'gdp_per_capita': 21774.768346769382},
 'SI': {'population': 2066880, 'gdp_per_capita': 30053.882187161325},
 'RO': {'population': 19053815, 'gdp_per_capita': 15811.089953586723},
 'BG': {'population': 7000039, 'gdp_per_capita': 12719.986046649168},
 'HR': {'population': 3871833, 'gdp_per_capita': 18328.426475263783},
 'CY': {'population': 1141166, 'gdp_per_capita': 24921.04806925548},
 'MT': {'population': 465292, 'gdp_per_capita': 38180.90578604403},
 'NL': {'population': 17100715, 'gdp_per_capita': 48569.46734975701}}

In [7]:
from auto_co2.agg import Countries
countries = Countries(df, country_stats)
countries.display_sorted(by='population')

,Country,population,gdp_per_capita,Count,Make,FuelType,FuelConsumption,EnginePower,Co2EmissionsWltp,MassRunningOrder,BaseWheel,AxleWidthSteering
5,DE,83149300.000,51232.360,1248,VOLKSWAGEN,PETROL,6.298,116.161,114.676,1569.635,2686.754,1548.603
10,FR,67749632.000,43658.979,926,RENAULT,PETROL,5.238,87.067,108.481,1404.463,2634.889,1539.748
15,IT,58850717.000,35927.443,733,FIAT,PETROL,4.709,83.173,121.253,1367.308,2577.690,nan
8,ES,47415750.000,30103.514,458,VOLKSWAGEN,PETROL,5.502,93.290,126.851,1418.474,2646.258,1550.943
21,PL,38382576.000,17929.401,204,TOYOTA,PETROL,4.984,106.475,134.549,1494.196,2679.113,1564.108
23,RO,19053815.000,15811.090,73,DACIA,PETROL,nan,89.768,122.329,1364.315,2655.918,1540.740
20,NL,17100715.000,48569.467,156,VOLKSWAGEN,PETROL,nan,104.147,90.705,1500.109,2660.000,1547.179
1,BE,11584008.000,51785.185,212,BMW,PETROL,5.057,111.397,114.924,1552.811,2686.175,1565.231
4,CZ,10827529.000,23152.189,116,SKODA,PETROL,6.001,113.310,141.336,1461.069,2672.681,1550.491
24,SE,10548336.000,60375.043,127,VOLVO,PETROL,nan,134.118,85.606,1667.685,2718.764,1571.835


In [8]:
co2.viz.countrywise_viz(countries)

In [9]:
from auto_co2.agg import Manufacturers
makers = Manufacturers(df)
makers.display()

In [10]:
co2.viz.plot_registrations_per_month(df)

In [11]:
co2.styles.display_info(df)

,Column,Non-Null Count,Dtype
0,ID,5000 non-null,int64
1,Country,5000 non-null,object
2,Pool,4838 non-null,object
3,ManufacturerName,5000 non-null,object
4,ManufNameOem,5000 non-null,object
5,Make,5000 non-null,object
6,CategoryOf,5000 non-null,object
7,MassRunningOrder,5000 non-null,float64
8,WltpTestMass,4034 non-null,float64
9,Co2EmissionsWltp,4949 non-null,float64


,Info
0,"RangeIndex: 5000 entries, 0 to 4999"
1,Data columns (total 23 columns):
2,22 Month 4485 non-null float64
3,"dtypes: datetime64[ns](1), float64(12), int64(1), object(9)"
4,memory usage: 898.6+ KB


In [12]:
co2.styles.display_na(df)

,Colonne,Valeurs manquantes (%)
0,ElectricRange,85.46%
1,ElectricConsumption,84.24%
2,InnovativeEmissionsReductionWltp,56.10%
3,InnovativeTechnology,53.26%
4,FuelConsumption,40.68%
5,WltpTestMass,19.32%
6,AxleWidthSteering,15.72%
7,Month,10.30%
8,RegistrationDate,10.30%
9,EngineCapacity,10.16%


In [13]:
co2.styles.display_describe(df)

,Colonne,count,mean,std,min,25%,50%,75%,max
0,ID,5000.000000,47122714.080200,458.000000,46640799.000000,51717548.000000,54894135.000000,61299368.000000,15403641.486526
1,MassRunningOrder,5000.000000,1483.880600,915.000000,1243.000000,1421.000000,1658.000000,3298.000000,335.908187
2,WltpTestMass,4034.000000,1614.263014,978.000000,1359.000000,1552.000000,1793.000000,3344.000000,358.144636
3,Co2EmissionsWltp,4949.000000,114.908668,0.000000,111.000000,125.000000,141.000000,373.000000,52.866431
4,BaseWheel,4990.000000,2655.320240,1873.000000,2567.000000,2646.000000,2720.000000,4490.000000,169.843876
5,AxleWidthSteering,4214.000000,1551.089464,1365.000000,1514.000000,1558.000000,1585.000000,2539.000000,61.429044
6,EngineCapacity,4492.000000,1518.385129,875.000000,1084.000000,1496.000000,1950.000000,6749.000000,503.034762
7,EnginePower,4945.000000,101.822042,10.000000,70.000000,95.000000,112.000000,460.000000,50.052677
8,ElectricConsumption,788.000000,171.602792,45.000000,150.000000,163.000000,180.000000,373.000000,37.208617
9,InnovativeEmissionsReductionWltp,2195.000000,1.519043,0.500000,1.000000,1.600000,1.900000,4.300000,0.511884


In [14]:
co2.viz.plot_fueltype_distribution(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID                                5000 non-null   int64         
 1   Country                           5000 non-null   object        
 2   Pool                              4838 non-null   object        
 3   ManufacturerName                  5000 non-null   object        
 4   ManufNameOem                      5000 non-null   object        
 5   Make                              5000 non-null   object        
 6   CategoryOf                        5000 non-null   object        
 7   MassRunningOrder                  5000 non-null   float64       
 8   WltpTestMass                      4034 non-null   float64       
 9   Co2EmissionsWltp                  4949 non-null   float64       
 10  BaseWheel                         4990 non-null 

In [16]:
import plotly.graph_objects as go
import colorlover as cl


def plot_popular_fueltype(manufacturers:Manufacturers, save=True, format='png'):
    # Group by Pool and Make and sum the Counts for each group
    grouped_df = manufacturers.data.groupby(['Pool', 'Make'])['Count'].sum().reset_index(name='Counts')

    # Create a list of traces
    traces = []
    colors = cl.scales['12']['qual']['Paired']  # Get a list of 12 distinct colors
    make_colors = {make: colors[i % len(colors)] for i, make in enumerate(grouped_df['Make'].unique())}  # Create a color map
    
    for make in grouped_df['Make'].unique():
        df = grouped_df[grouped_df['Make'] == make]
        traces.append(go.Bar(x=df['Pool'], y=df['Counts'], name=make, marker_color=make_colors[make]))

    layout = go.Layout(barmode='stack', title="Répartition des types de carburants par marque/groupe automobile")
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

    if save:
        save_plotly_fig(fig, format)

In [17]:
import plotly.graph_objects as go
plot_popular_fueltype(makers,save=False)
